In [1]:
import os
current_directory = os.getcwd()
import sys; sys.path.insert(0, current_directory)

In [2]:
import mne
import pickle as pkl

from time import time
import os
import ot
import numpy as np
from invert.util import pos_from_forward
from scipy.spatial.distance import cdist
import time  
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

#### =========================================================================
#### Section 1: Earth Mover's Distance (EMD) Evaluation Function
#### This function calculates the EMD between two source distributions. EMD,
#### also known as the Wasserstein distance, measures the "work" required to
#### transform one distribution into the other, providing a spatial similarity
#### metric between the true and estimated brain activity.
#### =========================================================================

In [3]:
def eval_emd(distances, values_1, values_2):
    values_1 = abs(values_1).mean(axis=-1)
    values_2 = abs(values_2).mean(axis=-1)
    
    values_1 = values_1 / np.sum(values_1)
    values_2 = values_2 / np.sum(values_2)    
    
    emd_value = ot.emd2(values_1, values_2, distances)
    return emd_value

#### =========================================================================
#### Section 2: Load Simulation & System Configuration
#### This block sets up the environment by loading the forward model that was
#### used to generate the original data. It also defines a helper function to
#### save the final results and creates a directory to store them.
#### =========================================================================

In [4]:
file_path = "forward_models"
mult = 2
folder_path = os.path.join(file_path, "128_ch_coarse_80_ratio-fwd.fif")
folder_path

'forward_models\\128_ch_coarse_80_ratio-fwd.fif'

In [5]:
fwd_for = mne.read_forward_solution(folder_path, verbose=0)
fwd_for = mne.convert_forward_solution(fwd_for, force_fixed=True)
pos_for = pos_from_forward(fwd_for)

fn = os.path.join(file_path, "128_ch_info.fif")
info = mne.io.read_info(fn)

    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]


In [6]:
def save_data(data_dict, folder_path, filename):
    file_path = os.path.join(folder_path, filename)
    with open(file_path, 'wb') as f:
        pkl.dump(data_dict, f)

def load_data(file_path):
    with open(file_path, 'rb') as f:
        data = pkl.load(f)
    return data

In [7]:
folder_save = "Results"
folder_pathsave = os.path.join(os.getcwd(), folder_save)
os.makedirs(folder_pathsave, exist_ok=True)

#### =========================================================================
#### Section 3: Load Inverse Model and Pre-computed Data
#### This section loads the forward model used for the inverse solution (to
#### avoid the 'inverse crime') and pre-calculated patch-based components
#### (`Weights` and `Lpatch_Fulls`) required for analysis. It also computes
#### the distance matrix between the source spaces of the two models.
#### =========================================================================

In [8]:
model_paths = {
    "coarse-80": r"128_ch_coarse_80_ratio-fwd.fif", # 5124 sasmpling points
    # "fine-80": r"128_ch_fine_80_ratio-fwd.fif", # 8196 sasmpling points done
    # "fine-50": r"128_ch_fine_50_ratio-fwd.fif",
    # "fine-20": r"128_ch_fine_20_ratio-fwd.fif"
    }
fwds = dict()
for inv_name, model_path in model_paths.items():
    fwd_inv = mne.read_forward_solution(os.path.join(file_path,model_path), verbose=0)
    fwd_inv = mne.convert_forward_solution(fwd_inv, surf_ori=True, force_fixed=True,use_cps=True, verbose=0) 
    fwds[inv_name] = fwd_inv
    pos_inv = pos_from_forward(fwd_inv)
distances = cdist(pos_for, pos_inv)

#### =========================================================================
#### Section 4: Set Up Data Loading Paths
#### This block defines a helper function to load data from pickle files and
#### specifies the directory paths for both the raw simulated data and the
#### previously processed (evaluated) inverse solutions.
#### =========================================================================

In [ ]:
leadfields = fwd_inv['sol']['data']               
n_dipoles = np.shape(leadfields)[1]    

file_path = ""

Weights = pkl.load(open(os.path.join(file_path, "KQ_MaxExtent_{}_{}.pkl".format(10, inv_name)), 'rb'))
Lpatch_Fulls = pkl.load(open(os.path.join(file_path, "ULpatch_MaxExtent_{}_MaxRank_{}_{}.pkl".format(10,5, inv_name)), 'rb'))  
Lpatch_Fulls = Lpatch_Fulls[mult]

In [10]:
folder_load = "Simulated_Data"
folder_pathload = os.path.join(os.getcwd(), folder_load)

# Define the folder path
folder_name = "Evaluated_Data"
folder_path = os.path.join(os.getcwd(), folder_name)

#### =========================================================================
#### Section 5: Main Evaluation Loop
#### This is the core of the script. It iterates through various simulation
#### conditions (correlation, smoothness, patch ranks, SNR). For each
#### condition, it loads the ground truth data and the corresponding inverse
#### solution results, then computes the EMD metric for each solver.
#### =========================================================================

In [11]:
batch_size = 50 # number of monte-carlo repettions 
plotmax = 1
# Patchranks_Full  = [[1],[2],[3],[1,1],[1,2],[1,3],[2,2],[2,3],[1,2,3]]  
# Patchranks_Full  = [[2],[1,2],[1,2,2]]   
Patchranks_Full = [[1,2]]  
# Patchranks_Full  = [[1,1]]   
for corr_coeff in [0.5]:
    for Smoothness_order in range(2,4,2):        
        for Patchranks in Patchranks_Full:   
            # Define lists to store EMD results for each SNR value                 
            for snr_db in range(-5,10,5):
                start_time = time.time()
                n_jobs = 5
                # Iterate through all files in the folder
                filename = f"Data_corr_{corr_coeff}_smooth_{Smoothness_order}_patchranks_{Patchranks}_snr_{snr_db}.pkl"
                file_path = os.path.join(folder_pathload, filename)
                loaded_data = load_data(file_path)
                Y = loaded_data["Y"]
                SSFull = loaded_data["SddotFull"]
                SS = [sum(sublist) for sublist in SSFull]
                Y = Y[:batch_size]
                # Iterate through all files in the folder
                filename = f"LenEvaluate_MError_{inv_name}_Data_corr_{corr_coeff}_smooth_{Smoothness_order}_patchranks_{Patchranks}_snr_{snr_db}.pkl"
                file_path = os.path.join(folder_path, filename)
                evaluated_data = load_data(file_path) 
                solver_names = evaluated_data["solver_names"]
                stc_dict = evaluated_data["STCs"]
                
                # =====================================================
                # Subsection 5.1: Calculate EMD in Parallel
                # This block uses a ThreadPoolExecutor to compute the EMD
                # for each sample in the batch and for each inverse
                # solver in parallel, significantly speeding up the
                # evaluation process.
                # =====================================================                                                                            
                def compute_emd_for_solver(solver_name, i, distances, SS, stc_dict):
                    print(f"{solver_name} of {i}")
                    print(f"SS of {i}", SS[i])
                    print(f"stc_dict of {solver_name} and {i}", stc_dict[solver_name][i]);
                    J_true = SS[i]
                    J_Pred = stc_dict[solver_name][i]
                    return eval_emd(distances, J_true, J_Pred)
                
                EMD_results = {}

                # Use ThreadPoolExecutor or ProcessPoolExecutor depending on the nature of your task
                import concurrent.futures
                with concurrent.futures.ThreadPoolExecutor() as executor:
                    future_to_solver = {solver_name: [] for solver_name in solver_names}
                    
                    for solver_name in solver_names:
                        futures = []
                        for i in range(batch_size):
                            print(f"{solver_name} of {i}")
                            future = executor.submit(compute_emd_for_solver, solver_name, i, distances, SS, stc_dict)
                            futures.append(
                                future
                            )
                            print(f"Inside future of '{solver_name}': '{future}'")
                        future_to_solver[solver_name] = futures
                        
                # for solver_name, futures in future_to_solver.items():
                #     EMD_results[solver_name] = [future.result() for future in futures]

                for solver_name, futures in future_to_solver.items():
                    safe_results = []
                    for future in futures:
                        try:
                            result = future.result()
                            safe_results.append(result)
                        except IndexError as e:
                            # Handle index out-of-range gracefully
                            print(f"[Warning] IndexError for solver '{solver_name}': {e}")
                            # safe_results.append(None)  # or use a default value
                        except Exception as e:
                            # Catch any other unexpected errors
                            print(f"[Warning] Error for solver '{solver_name}': {e}")
                            # safe_results.append(None)
                    EMD_results[solver_name] = safe_results

                    
                # =====================================================
                # Subsection 5.2: Save EMD Results
                # The computed EMD values for all solvers are saved to a
                # new file, creating a final record of this evaluation run.
                # =====================================================
                filename = f"EMD_Len_{inv_name}_Data_corr_{corr_coeff}_smooth_{Smoothness_order}_patchranks_{Patchranks}_snr_{snr_db}.pkl"
                save_data(EMD_results, folder_pathsave, filename)
                
                end_time = time.time()  # Record end time
                elapsed_time = end_time - start_time
                print(f"Elapsed time for corr_coeff={corr_coeff}, Smoothness_order={Smoothness_order}, Patchranks={Patchranks}, snr_db={snr_db}: {elapsed_time} seconds")

RAP-MUSIC of 0
RAP-MUSIC of 0Inside future of 'RAP-MUSIC': '<Future at 0x1acb81c55e0 state=running>'
RAP-MUSIC of 1
RAP-MUSIC of 1
SS of 1 Inside future of 'RAP-MUSIC': '<Future at 0x1acb6aa37a0 state=running>'
RAP-MUSIC of 2
RAP-MUSIC of 2
SS of 2 
SS of 0 Inside future of 'RAP-MUSIC': '<Future at 0x1acb7c49400 state=running>'
RAP-MUSIC of 3
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
stc_dict of RAP-MUSIC and 1 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
stc_dict of RAP-MUSIC and 2 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 

MemoryError: 